In [1]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
# from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from xvfbwrapper import Xvfb

display = Xvfb()
display.start()

options = webdriver.ChromeOptions()
options.add_argument("enable-automation")
options.add_argument("--headless")
options.add_argument("--window-size=800,600")
options.add_argument("no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.set_headless()

data = []
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
actions = ActionChains(driver)

('foo', 'foo')


In [ ]:
import threading
import queue

my_queue = queue.Queue()

def storeInQueue(f):
    def wrapper(*args):
        my_queue.put(f(*args))
    return wrapper


@storeInQueue
def get_anime(anim):
    if idx <= 233:
        continue

    xpath = anim.find_element_by_xpath('.//ul/li/a')
    title = xpath.text
    link = xpath.get_attribute('href')
    driver.execute_script('window.open("'+link+'","_blank");')
    driver.switch_to.window(driver.window_handles[1])
    title_detail = driver.find_element_by_class_name('jdlrx').find_element_by_tag_name('h1').text
    img = driver.find_element_by_class_name('fotoanime').find_element_by_tag_name('img').get_attribute('src')

    detail = driver.find_elements_by_xpath('.//div[@class="infozingle"]/p')
    title_det = detail[0].find_element_by_tag_name('span').text.split(":")[1]
    title_jpn = detail[1].find_element_by_tag_name('span').text.split(":")[1]
    skor = detail[2].find_element_by_tag_name('span').text.split(":")[1]
    produser = detail[3].find_element_by_tag_name('span').text.split(":")[1]
    tipe = detail[4].find_element_by_tag_name('span').text.split(":")[1]
    status = detail[5].find_element_by_tag_name('span').text.split(":")[1]
    total_eps = detail[6].find_element_by_tag_name('span').text.split(":")[1]
    durasi = detail[7].find_element_by_tag_name('span').text.split(":")[1]
    tgl_rilis = detail[8].find_element_by_tag_name('span').text.split(":")[1]
    studio = detail[9].find_element_by_tag_name('span').text.split(":")[1]
    genre = detail[10].find_element_by_tag_name('span').text.split(":")[1]

    print(">>>>>>>>>>>>",title)

    sinopsis = driver.find_elements_by_xpath('.//div[@class="sinopc"]/p')
    data_sinopsis = ""
    for sin in sinopsis:
        data_sinopsis += sin.text

    eps_anime = driver.find_elements_by_xpath('.//div[@class="episodelist"]')[1]
    eps_list = eps_anime.find_elements_by_xpath('.//ul/li')
    data_eps = []
    for eps in eps_list:
        eps_name = eps.find_element_by_xpath('.//span[1]/a').text
        url = eps.find_element_by_xpath('.//span[1]/a').get_attribute('href')
        date = eps.find_element_by_xpath('.//span[2]').text
        print(eps_name)

        driver.execute_script('window.open("'+url+'","_blank");')
        driver.switch_to.window(driver.window_handles[2])

        src_video = driver.find_element_by_xpath('//div[@class="responsive-embed-stream"]/iframe').get_attribute('src')
        video = (eps_name, url, date, src_video)

        data_eps.append(video)

        driver.close()
        driver.switch_to.window(driver.window_handles[1])

    anime = ('anime'+str(idx), title, title_detail, img, title_det, title_jpn, skor, produser, tipe, status, total_eps, durasi, tgl_rilis, studio, genre, data_sinopsis, data_eps)
    return anime


In [ ]:
driver.get('https://otakudesu.moe/anime-list')

In [ ]:
data = []
list_anime = driver.find_elements_by_class_name('jdlbar')
print(len(list_anime))
try:
    for idx, anim in enumerate(list_anime):
        if idx <= 500:
            continue

        t = threading.Thread(target=get_anime, args = (anim, ))
        t.start()
        
        data.append(my_queue.get())

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

In [ ]:
with open('otakudesu11.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['id', 'title', 'title_detail', 'img', 'title_ori', 'title_jpn', 'skor', 'produser', 'tipe', 'status', 'total_eps', 'durasi', 'tgl_rilis', 'studio', 'genre', 'data_sinopsis', 'data_eps']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)